> **License Notice**
> 
> This notebook is part of the *Computational Economics and Data Science* course.
> 
> - Code cells are released under the MIT License. You are free to use and adapt the code with attribution.
> - Text, figures, and other non-code content are released under the Creative Commons Attribution 4.0 International License.
> 
> Please retain this notice and credit Amirreza "Farnam" Taheri when sharing adaptations.

In [ ]:
# === Environment Setup ===
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats, linalg
from scipy.optimize import fsolve
import seaborn as sns
from IPython.display import Markdown, display

# --- Custom Display Functions ---
def theorem(title, statement):
    display(Markdown(f"""
    <div style='background-color:#e7f3e7; padding:15px; border-left:5px solid #4CAF50'>
    <strong>Theorem ({title}):</strong><br/>
    {statement}
    </div>
    """))

def example(title, content):
    display(Markdown(f"""
    <div style='background-color:#e3f2fd; padding:15px; border-left:5px solid #2196F3'>
    <strong>Example ({title}):</strong><br/>
    {content}
    </div>
    """))

def econ_app(title, content):
    display(Markdown(f"""
    <div style='background-color:#fff3e0; padding:15px; border-left:5px solid #FF9800'>
    <strong>💼 Economic Application ({title}):</strong><br/>
    {content}
    </div>
    """))

def course_connection(content):
    display(Markdown(f"""*📚 **Course Connection:** {content}*"""))

# --- Plotting Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 14, 'figure.figsize': (10, 6), 'figure.dpi': 150})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# Appendix A4: Linear Algebra

---
## Table of Contents

- 4.1 Matrix Operations
- 4.2 Special Matrices
- 4.3 Eigenvalues and Eigenvectors
- 4.4 Matrix Decompositions
- 4.5 Vector and Matrix Norms
- 4.6 Important Theorems (Summary)

**Historical Note:** While the concepts of determinants and solving systems of linear equations have been studied for centuries, the formalization of matrices and matrix algebra is largely a 19th-century development. The term "matrix" was coined by **James Joseph Sylvester** in 1850. **Arthur Cayley** developed the algebraic properties of matrices, including multiplication and inversion, making him a key figure in the founding of modern linear algebra.

**Purpose**: Linear algebra is the language of modern computation and data analysis. This section provides a condensed reference for the most common linear algebra operations, concepts, and theorems used in computational economics. We will cover the properties of vectors and matrices, the crucial concepts of eigenvalues and eigenvectors for understanding dynamic systems, and the matrix decompositions that form the backbone of numerical algorithms.

## 4.1 Matrix Operations

- **Transpose**: $(A^T)_{ij} = A_{ji}$. Key property: $(AB)^T = B^T A^T$.
- **Trace**: $\text{tr}(A) = \sum_{i} A_{ii}$. Key property: $\text{tr}(AB) = \text{tr}(BA)$.
- **Determinant**: A scalar value that provides information about the matrix. Key property: $\det(AB) = \det(A)\det(B)$. A matrix is invertible if and only if its determinant is non-zero.
- **Inverse**: For a square matrix $A$, its inverse $A^{-1}$ is the matrix such that $AA^{-1} = A^{-1}A = I$. For solving linear systems $Ax=b$, it is almost always numerically better to use a solver like `np.linalg.solve(A, b)` than to compute the inverse directly.

## 4.2 Special Matrices

- **Symmetric**: $A = A^T$. Symmetric matrices always have real eigenvalues and orthogonal eigenvectors.
- **Diagonal**: All off-diagonal elements are zero.
- **Identity ($I$)**: A diagonal matrix with all diagonal elements equal to 1.
- **Orthogonal**: $Q^T Q = Q Q^T = I$, which implies $Q^{-1} = Q^T$. Orthogonal matrices represent rotations and reflections and preserve lengths (i.e., $\|Qx\| = \|x\|$).
- **Positive Definite (PD)**: A symmetric matrix $A$ is PD if $x^T A x > 0$ for all non-zero vectors $x$. This is equivalent to all its eigenvalues being positive. Covariance matrices are a key example.

## 4.3 Eigenvalues and Eigenvectors

For a square matrix $A$, an **eigenvector** $v$ is a non-zero vector that does not change its direction when the linear transformation $A$ is applied to it. It is 'invariant' in direction. It is only stretched or shrunk by a scalar factor $\lambda$, called the **eigenvalue**. Formally, the defining relationship is:
$$Av = \lambda v$$**Geometric Intuition**: Most vectors will be rotated and scaled by a matrix transformation. Eigenvectors are special because they lie on the 'axes of transformation'—the directions that are preserved. The corresponding eigenvalue tells us how much the eigenvector is scaled along that axis. A positive eigenvalue means it's stretched in the same direction, a negative eigenvalue means it's flipped and stretched, and an eigenvalue between -1 and 1 means it's shrunk.

The plot below illustrates the concept of eigenvectors. The transformation matrix A is applied to three vectors. The eigenvectors ($v_1$ and $v_2$) do not change their direction; they are only scaled by their corresponding eigenvalue. Their transformed versions ($Av_1$ and $Av_2$) lie on the same line as the original vectors. In contrast, the non-eigenvector ($v_{other}$) changes its direction after the transformation.

![Eigenvector Visualization](../images/png/eigenvectors.png)
*<center>This image was programmatically generated for the course.</center>*

In [ ]:
theorem("Spectral Theorem for Symmetric Matrices", "If $A$ is a real symmetric matrix, then it can be factored as $A = Q\\Lambda Q^T$, where $Q$ is an orthogonal matrix whose columns are the eigenvectors of $A$, and $\\Lambda$ is a diagonal matrix whose entries are the corresponding eigenvalues.")

**Motivation**: Eigenvalues and eigenvectors are crucial for understanding the behavior of **linear dynamic systems**, which are ubiquitous in economics. For a system described by $x_{t+1} = A x_t$, the eigenvalues of the matrix $A$ determine the stability of the system. If all eigenvalues have a magnitude less than 1, the system is stable and will converge to a steady state. If any eigenvalue has a magnitude greater than 1, the system is explosive. The eigenvectors represent the stable or unstable directions of the system. This analysis is fundamental to macroeconomics (e.g., solving DSGE models) and time series econometrics. Eigenvalues are also the basis for dimensionality reduction techniques like Principal Component Analysis (PCA), which is widely used in finance and data science to extract the most important 'factors' from high-dimensional data.

## 4.4 Matrix Decompositions

Matrix decompositions are ways of rewriting a matrix as a product of simpler matrices. They are the workhorses of numerical linear algebra.
- **LU Decomposition**: $A = LU$. Used for efficiently solving $Ax=b$.
- **QR Decomposition**: $A = QR$. The most stable way to solve least-squares problems.
- **Cholesky Decomposition**: $A = LL^T$. A fast and efficient decomposition for positive definite matrices, used in simulating correlated random variables.
- **Eigenvalue Decomposition (Spectral)**: $A = Q\Lambda Q^T$. Used for understanding dynamics and for diagonalizing quadratic forms.
- **Singular Value Decomposition (SVD)**: $A = U\Sigma V^T$. The most general decomposition. It always exists and is the basis for Principal Component Analysis (PCA).

In [ ]:
econ_app("Principal Component Analysis (PCA)", "PCA uses the eigenvalue decomposition of a covariance matrix to find the directions of maximum variance in the data. These directions, the principal components, can be used for dimensionality reduction. This is widely used in factor models in finance."),
course_connection("Eigenvalues determine the stability of dynamic systems (Chapter 4). Decompositions are used for numerical linear algebra (Chapter 2.1) and OLS (Chapter 6.1).")

## 4.5 Vector and Matrix Norms

Norms are functions that measure the length or size of vectors and matrices.

**Vector Norms:**
- **$\ell_1$ norm (Manhattan distance)**: $\\|x\\|_1 = \\sum_i |x_i|$.
- **$\ell_2$ norm (Euclidean distance)**: $\\|x\\|_2 = \\sqrt{\\sum_i x_i^2}$.
- **$\ell_\\infty$ norm (Max norm)**: $\\|x\\|_\\infty = \\max_i |x_i|$.

**Matrix Norms:**
- **Frobenius norm**: $\\|A\\|_F = \\sqrt{\\sum_{i,j} A_{ij}^2}$.
- **Operator norm (or spectral norm)**: $\\|A\\|_2 = \\sigma_{max}(A)$, the largest singular value of A.

## 4.6 Important Theorems (Summary)

- **Rank-Nullity Theorem**: For any $m \times n$ matrix $A$, the dimension of its column space (rank) plus the dimension of its null space (nullity) equals the number of columns, $n$.
- **Cayley-Hamilton Theorem**: Every square matrix satisfies its own characteristic equation.
- **Courant-Fischer Min-Max Theorem**: Provides a variational characterization of the eigenvalues of a symmetric matrix.
- **Perron-Frobenius Theorem**: For a square matrix with positive entries, there is a unique largest real eigenvalue (the 'Perron root'). This eigenvalue is positive, and its corresponding eigenvector has strictly positive components. This theorem is fundamental for the stability of linear dynamic systems and Markov chains.

In [ ]:
econ_app("Dynamic Systems and Stationary Distributions", "The Perron-Frobenius theorem is fundamental to the study of linear dynamic systems with positive components, such as multi-sector growth models or Markov chains. For an irreducible and aperiodic Markov chain, its transition matrix has a unique largest eigenvalue equal to 1. The theorem guarantees this. The corresponding eigenvector, when normalized to be a probability vector (its elements are positive and sum to 1), is the **unique stationary distribution** of the Markov chain. This distribution describes the long-run proportion of time the system will spend in each state, regardless of its starting state. This is a cornerstone result for many models in macroeconomics, finance, and industrial organization.")